<a href="https://colab.research.google.com/github/siva4iov/FlatsPrice_DS_Proj/blob/master/Geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geocoding

### Dependencies

In [6]:
!pip install geopy
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import os

tqdm.pandas()

In [8]:
if not os.path.exists("/content/dataset_clean.csv"):
  from google.colab import files
  files.upload()

### Addresses


In [76]:
df = pd.read_csv("/content/dataset_clean.csv", sep=";")
ads = df.loc[:, "Address"]
ads.head()


0    Республика Татарстан, Казань, ул. Разведчика А...
1    Республика Татарстан, Казань, Вахитовский райо...
2    Республика Татарстан, Казань, ул. Мидхата Була...
3    Республика Татарстан, Казань, пр-т Альберта Ка...
4    Республика Татарстан, Казань, ул. Сибгата Хаки...
Name: Address, dtype: object

### I will use only free geocoding services

In [10]:
geolocator = Nominatim(user_agent="my_request")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
data = ads.progress_apply(geocode)

100%|██████████| 1606/1606 [35:49<00:00,  1.34s/it]


In [77]:
location = data.rename("location")

In [78]:
coord = pd.Series(list(map(lambda x: tuple(x.point)[:2] if x else None, data)), name="coord")

In [79]:
geodata = pd.concat([location, coord], axis=1)

I will drop addresses which were not found

In [82]:
df = df.loc[geodata.notna().iloc[:, 0]].reset_index(drop=True)

geodata = geodata.dropna().reset_index(drop=True)


### Extract district

In [83]:
def extr_distr(x):
  objs = str(x).split(",")
  for obj in objs:
    if "район" in obj:
      result = obj
      result = result.replace("район", "").strip()
      return result

geodata.loc[:, "district"] = geodata["location"].apply(extr_distr)

In [84]:
geodata

,location,coord,district
0,"(Газпромбанк, 3, улица Разведчика Ахмерова, Ar...","(55.7978535, 49.1793271)",Советский
1,"(53, улица Татарстан, Старо-Татарская слобода,...","(55.77585095, 49.1077901578817)",Вахитовский
2,"(5, улица Мидхата Булатова, 33-й Военный город...","(55.731757599999995, 49.169174993589834)",Приволжский
3,"(31, улица Сибгата Хакима, Ново-Савиновский ра...","(55.816227350000005, 49.13680551681148)",Ново-Савиновский
4,"(Ozon, 13, улица Адоратского, Ново-Савиновский...","(55.8252666, 49.1468802)",Ново-Савиновский
...,...,...,...
1318,"(улица Чернышевского, Вахитовский район, Казан...","(55.7930993, 49.1107986)",Вахитовский
1319,"(12, улица Волкова, Аки, Советский район, Каза...","(55.84985115, 49.25372233475784)",Советский
1320,"(Пятёрочка, 14В, улица Юлиуса Фучика, Горки-3,...","(55.7420827, 49.2053247)",Приволжский
1321,"(61Б, Чистопольская улица, Ново-Савиновский ра...","(55.82022285, 49.13268965)",Ново-Савиновский


## Also we need to get rid of cyrillic to avoid future difficulties
But, i will leave column location for now, because it could be useful in future.

I will drop it later

In [85]:
condition = {dist: num+1 for num, dist in enumerate(geodata['district'].unique())}
display(condition)
districts = \
pd.get_dummies(geodata.replace({"district": condition})["district"], prefix="district")
geodata = pd.concat([geodata.drop(columns=["district"]), districts], axis=1)

{'Советский': 1,
 'Вахитовский': 2,
 'Приволжский': 3,
 'Ново-Савиновский': 4,
 'Московский': 5,
 'Кировский': 6,
 'Авиастроительный': 7,
 'микро М-14': 8,
 'РУВД Московского а': 9}

In [88]:
df = pd.concat([df.drop(columns=["Address"]), geodata], axis=1)

# Saving dataset and new features meaning

In [90]:
df.to_csv("dataset_clean_full.csv", sep=";", index=False)

In [94]:
features_decode = pd.read_csv("/content/features_decode.csv", sep=";")

In [101]:
meanings = {'Советский': 'district_1',
 'Вахитовский': 'district_2',
 'Приволжский': 'district_3',
 'Ново-Савиновский': 'district_4',
 'Московский': 'district_5',
 'Кировский': 'district_6',
 'Авиастроительный': 'district_7',
 'микро М-14': 'district_8',
 'РУВД Московского а': 'district_9'}

meanings = {'name': list(meanings.keys()), 'meaning': list(meanings.values())}
meanings = pd.DataFrame(meanings)
meanings

,name,meaning
0,Советский,district_1
1,Вахитовский,district_2
2,Приволжский,district_3
3,Ново-Савиновский,district_4
4,Московский,district_5
5,Кировский,district_6
6,Авиастроительный,district_7
7,микро М-14,district_8
8,РУВД Московского а,district_9


In [104]:
features_decode = pd.concat([features_decode, meanings]).reset_index(drop=True)

In [105]:
features_decode.to_csv("features_decode.csv", sep=";")